In [ ]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import plotly.express as px

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
#@title Default title text
host = "ec2-54-164-22-242.compute-1.amazonaws.com"
dbname = "d92sarup3qlji0"
user = "fttxsdmckdfikz"
password = "c5561e667cd292f382a0e437807e05c89fba1b894dddff1e77e07befe8a866ac"

In [ ]:
connection = pg.connect("host={} dbname={} user={} password={}".format(host, dbname, user, password))
df_orders = psql.read_sql('SELECT * FROM orders', connection)
df_order_details = psql.read_sql_query('select * from order_details', connection)
df_products = psql.read_sql_query('select * from products', connection)
df_customers = psql.read_sql_query('select * from customers', connection)

In [ ]:
df_customers.head()

,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67


In [ ]:
df_orders.head()

,order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium


In [ ]:
df_order_details.head()

,order_id,product_id,unit_price,quantity,discount
0,10248,11,14.0,12,0.0
1,10248,42,9.8,10,0.0
2,10248,72,34.8,5,0.0
3,10249,14,18.6,9,0.0
4,10249,51,42.4,40,0.0


In [ ]:
# Recency = Max(order date)
# Frequency = count(customer id)
# Monetary = (unit price * quantity) - discount

In [ ]:
df_RFM = df_customers.merge(df_orders, left_on="customer_id", right_on="customer_id", how="inner")

In [ ]:
df_RFM.head()

,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax,order_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,10643,6,1997-08-25,1997-09-22,1997-09-02,1,29.46,Alfreds Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
1,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,10692,4,1997-10-03,1997-10-31,1997-10-13,2,61.02,Alfred's Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
2,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,10702,4,1997-10-13,1997-11-24,1997-10-21,1,23.94,Alfred's Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
3,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,10835,1,1998-01-15,1998-02-12,1998-01-21,3,69.53,Alfred's Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
4,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,10952,1,1998-03-16,1998-04-27,1998-03-24,1,40.42,Alfred's Futterkiste,Obere Str. 57,Berlin,None,12209,Germany


In [ ]:
df_RFM = df_RFM.merge(df_order_details, left_on="order_id", right_on="order_id", how="inner")

# Join customers >< orders >< order_details

In [ ]:
df_RFM.head()

,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax,order_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country,product_id,unit_price,quantity,discount
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,10643,6,1997-08-25,1997-09-22,1997-09-02,1,29.46,Alfreds Futterkiste,Obere Str. 57,Berlin,None,12209,Germany,28,45.6,15,0.25
1,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,10643,6,1997-08-25,1997-09-22,1997-09-02,1,29.46,Alfreds Futterkiste,Obere Str. 57,Berlin,None,12209,Germany,39,18.0,21,0.25
2,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,10643,6,1997-08-25,1997-09-22,1997-09-02,1,29.46,Alfreds Futterkiste,Obere Str. 57,Berlin,None,12209,Germany,46,12.0,2,0.25
3,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,10692,4,1997-10-03,1997-10-31,1997-10-13,2,61.02,Alfred's Futterkiste,Obere Str. 57,Berlin,None,12209,Germany,63,43.9,20,0.00
4,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545,10702,4,1997-10-13,1997-11-24,1997-10-21,1,23.94,Alfred's Futterkiste,Obere Str. 57,Berlin,None,12209,Germany,3,10.0,6,0.00


# Feature Engineering -> Extract R, F, M

In [ ]:
df_features = pd.DataFrame(df_RFM.groupby("customer_id")["order_date"].max())

In [ ]:
df_features.head()

,customer_id,order_date
0,ALFKI,1998-04-09
1,ANATR,1998-03-04
2,ANTON,1998-01-28
3,AROUT,1998-04-10
4,BERGS,1998-03-04


In [ ]:
df_features = df_features.reset_index()

In [ ]:
df_features.head()

,customer_id,order_date
0,ALFKI,1998-04-09
1,ANATR,1998-03-04
2,ANTON,1998-01-28
3,AROUT,1998-04-10
4,BERGS,1998-03-04


## Recency

In [ ]:
df_features["recency"] = df_features.apply(lambda x:pd.to_datetime("1999-01-01") - pd.to_datetime(x["order_date"]), axis=1 )

In [ ]:
df_features["recency"] = df_features.recency.dt.days

In [ ]:
df_features.head()

,customer_id,order_date,recency
0,ALFKI,1998-04-09,267
1,ANATR,1998-03-04,303
2,ANTON,1998-01-28,338
3,AROUT,1998-04-10,266
4,BERGS,1998-03-04,303


## Frequency

In [ ]:
df_RFM.groupby(by="customer_id")["customer_id"].count()

customer_id
ALFKI    12
ANATR    10
ANTON    17
AROUT    30
BERGS    52
         ..
WARTH    37
WELLI    19
WHITC    40
WILMK    17
WOLZA    16
Name: customer_id, Length: 89, dtype: int64

In [ ]:
df_temp_frequency = pd.DataFrame(df_RFM.groupby(by="customer_id")["order_id"].count())

In [ ]:
df_features = df_features.merge(df_temp_frequency, left_on="customer_id", right_on="customer_id", how="inner")

In [ ]:
df_features.frequency = df_features.order_id
del df_features["order_id"]
df_features

,customer_id,order_date,recency,frequency
0,ALFKI,1998-04-09,267,12
1,ANATR,1998-03-04,303,10
2,ANTON,1998-01-28,338,17
3,AROUT,1998-04-10,266,30
4,BERGS,1998-03-04,303,52
...,...,...,...,...
84,WARTH,1998-04-15,261,37
85,WELLI,1998-03-09,298,19
86,WHITC,1998-05-01,245,40
87,WILMK,1998-04-07,269,17


## Monetary

In [ ]:
df_RFM["monetary_temp"] = (df_RFM["unit_price"] * df_RFM["quantity"]) - df_RFM["discount"]

In [ ]:
df_temp_monetary = pd.DataFrame(df_RFM.groupby("customer_id")["monetary_temp"].sum()).reset_index()

In [ ]:
df_temp_monetary.head()

,customer_id,monetary_temp
0,ALFKI,4595.15
1,ANATR,1402.95
2,ANTON,7514.35
3,AROUT,13805.80
4,BERGS,26965.15


In [ ]:
df_features =  df_features.merge(df_temp_monetary, left_on="customer_id", right_on="customer_id", how="inner")

In [ ]:
df_features.columns = ["customer_id",	"order_date",	"recency",	"frequency",	"monetary"]

In [ ]:
df_features.head()

,customer_id,order_date,recency,frequency,monetary
0,ALFKI,1998-04-09,267,12,4595.15
1,ANATR,1998-03-04,303,10,1402.95
2,ANTON,1998-01-28,338,17,7514.35
3,AROUT,1998-04-10,266,30,13805.80
4,BERGS,1998-03-04,303,52,26965.15


# Binning

In [ ]:
df_features.recency.quantile([0.25,0.5,0.75])

0.25    248.0
0.50    263.0
0.75    298.0
Name: recency, dtype: float64

In [ ]:
px.box(df_features, y="recency")

In [ ]:
df_features.frequency.quantile([0.25,0.5,0.75])

0.25    11.0
0.50    20.0
0.75    31.0
Name: frequency, dtype: float64

In [ ]:
px.box(df_features, y="frequency")

In [ ]:
df_features.monetary.quantile([0.25,0.5,0.75])

0.25     3361.00
0.50     7553.80
0.75    18136.35
Name: monetary, dtype: float64

In [ ]:
px.box(df_features, y="monetary")

# Konversi Variabel R-F-M ke bin 1,2,3,4

In [ ]:
def convert_R_score(recency):
  # 0.25    248.0
  # 0.50    263.0
  # 0.75    298.0
  if recency >=0 and recency <= 248:
    return 1
  elif recency >248 and recency <= 263:
    return 2
  elif recency >263 and recency <= 298:
    return 3
  else:
    return 4

In [ ]:
df_features["R_score"] = df_features.apply(lambda x:convert_R_score(x["recency"]), axis=1)

In [ ]:
df_features.head()

,customer_id,order_date,recency,frequency,monetary,R_score
0,ALFKI,1998-04-09,267,12,4595.15,3
1,ANATR,1998-03-04,303,10,1402.95,4
2,ANTON,1998-01-28,338,17,7514.35,4
3,AROUT,1998-04-10,266,30,13805.80,3
4,BERGS,1998-03-04,303,52,26965.15,4
